# Bölüm 7: Örnekleri Kullanma (Birkaç Atışla Yönlendirme)

- [Ders](#lesson)
- [Alıştırmalar](#exercises)
- [Oyun Alanı](#example-playground)

## Kurulum

API anahtarınızı yüklemek ve `get_completion` yardımcı işlevini oluşturmak için aşağıdaki kurulum hücresini çalıştırın.

In [1]:
# Bu satırı sadece bir kere çalıştırmanız gerekiyor
%pip install -q -U google-generativeai 

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
# Import python's built-in regular expression library
import re
import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold

genai.configure(api_key="your-api-key")

# Create the model
generation_config = {
  "temperature": 0,
  "top_p": 1,
  "top_k": 1,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

def get_completion(chat, system_prompt="NONE", prefill="NONE"):
  asistant = genai.GenerativeModel(
    model_name="gemini-1.5-pro",
    generation_config=generation_config,
    safety_settings={
          HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
          HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
          HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
          HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE
    },
    system_instruction= system_prompt,
  )

  chat_session = asistant.start_chat(
    history=[
      {
        "role": "user",
        "parts": [chat],
      },
      {
        "role": "model",
        "parts": [prefill],
      },
    ]
  )
  response = chat_session.send_message(chat)
  return response.text

---

## Ders

**Gemini'a nasıl davranmasını istediğinize (ya da nasıl davranmamasını istediğinize) dair örnekler vermek son derece etkilidir**. Burada beklenen kritik noktalar:
- Doğru cevabı almak,
- Cevabı doğru formatta almaktır

Bu tür yönlendirme "**birkaç atışla (few shot) yönlendirme**" olarak da adlandırılır. Ayrıca "zero-shot" veya "n-shot" veya "one-shot" ifadeleriyle de karşılaşabilirsiniz. "Atış" sayısı, prompt içinde kaç örnek kullanıldığını ifade eder.

### Örnekler

Çocuklardan gelen sorulara yanıt veren bir "ebeveyn botu" oluşturmaya çalışan bir geliştirici olduğunuzu varsayın. Biliyorsunuz ki normalde Gemini'ın varsayılan yanıtı **oldukça resmi ve robotiktir**. Bu bir çocuğun kalbini kıracak seviyede soğuk yaklaşımlı ve fazla realist gelebilir.

In [14]:
# Prompt
PROMPT = "Noel Baba gerçek midir?"

# Print Gemini's response
print(get_completion(PROMPT))

Noel Baba, cömertliği ve neşeyi temsil eden büyülü bir figürdür! Çoğu yetişkin Noel Baba'nın, çocuklara nezaket, cömertlik ve hayal gücünün önemini öğretmek için yaratılmış bir hikaye olduğunu kabul eder. 

Noel Baba'nın kökenleri, çocuklara gizlice hediyeler veren gerçek bir kişi olan Aziz Nikolaos'a dayanmaktadır. Noel Baba efsanesi yüzyıllar boyunca, özellikle de yazarlar ve şairler tarafından şekillendirilerek bugün bildiğimiz neşeli ve cömert karaktere dönüşmüştür. 

Önemli olan Noel Baba'nın gerçek olup olmadığı değil, onun temsil ettiği şeydir: neşe, cömertlik ve sevdiklerimize değer verme ruhu. 



İstediğiniz tonu tarif etmek için zaman ayırabilirsiniz, ancak Gemini'a **ideal yanıtlardan birkaç örnek vermek**, istediğimiz tonu oluşturmayı çok daha kolaylaştıracaktır.

In [17]:
# Prompt
PROMPT = """Lütfen bir sonraki satırı “A” olarak yazarak konuşmayı tamamlayın.
Q: Diş perisi gerçek mi?
A: Elbette tatlım. Dişini paketle ve bu gece yastığının altına koy. Sabah seni bekleyen bir şey olabilir.
Q: Noel Baba gerçek midir?"""

# Print Gemini's response
print(get_completion(PROMPT))

A: Noel Baba'nın büyüsü gerçektir ve inanırsan, o da senin için gerçektir. 



Aşağıdaki biçimlendirme örneğinde, Gemini'a isimleri ve meslekleri nasıl ayıklayacağına ve sonra bunları tam olarak istediğimiz şekilde nasıl biçimlendireceğine dair bir dizi biçimlendirme talimatını adım adım gösterebilir veya sadece **Gemini'a doğru biçimlendirilmiş bazı örnekler sağlayabiliriz ve Gemini bu çrneklerden çıkarım yapabilir**. Gemini'ı doğru adımla başlatmak için `asistan` sırasındaki `<individuals>` parametresine dikkat edin.

In [18]:
# Prompt template with a placeholder for the variable content
PROMPT = """Silvermist Hollow, a charming village, was home to an extraordinary group of individuals.
Among them was Dr. Liam Patel, a neurosurgeon who revolutionized surgical techniques at the regional medical center.
Olivia Chen was an innovative architect who transformed the village's landscape with her sustainable and breathtaking designs.
The local theater was graced by the enchanting symphonies of Ethan Kovacs, a professionally-trained musician and composer.
Isabella Torres, a self-taught chef with a passion for locally sourced ingredients, created a culinary sensation with her farm-to-table restaurant, which became a must-visit destination for food lovers.
These remarkable individuals, each with their distinct talents, contributed to the vibrant tapestry of life in Silvermist Hollow.
<individuals>
1. Dr. Liam Patel [NEUROSURGEON]
2. Olivia Chen [ARCHITECT]
3. Ethan Kovacs [MISICIAN AND COMPOSER]
4. Isabella Torres [CHEF]
</individuals>

At the heart of the town, Chef Oliver Hamilton has transformed the culinary scene with his farm-to-table restaurant, Green Plate. Oliver's dedication to sourcing local, organic ingredients has earned the establishment rave reviews from food critics and locals alike.
Just down the street, you'll find the Riverside Grove Library, where head librarian Elizabeth Chen has worked diligently to create a welcoming and inclusive space for all. Her efforts to expand the library's offerings and establish reading programs for children have had a significant impact on the town's literacy rates.
As you stroll through the charming town square, you'll be captivated by the beautiful murals adorning the walls. These masterpieces are the work of renowned artist, Isabella Torres, whose talent for capturing the essence of Riverside Grove has brought the town to life.
Riverside Grove's athletic achievements are also worth noting, thanks to former Olympic swimmer-turned-coach, Marcus Jenkins. Marcus has used his experience and passion to train the town's youth, leading the Riverside Grove Swim Team to several regional championships.
<individuals>
1. Oliver Hamilton [CHEF]
2. Elizabeth Chen [LIBRARIAN]
3. Isabella Torres [ARTIST]
4. Marcus Jenkins [COACH]
</individuals>

Oak Valley, a charming small town, is home to a remarkable trio of individuals whose skills and dedication have left a lasting impact on the community.
At the town's bustling farmer's market, you'll find Laura Simmons, a passionate organic farmer known for her delicious and sustainably grown produce. Her dedication to promoting healthy eating has inspired the town to embrace a more eco-conscious lifestyle.
In Oak Valley's community center, Kevin Alvarez, a skilled dance instructor, has brought the joy of movement to people of all ages. His inclusive dance classes have fostered a sense of unity and self-expression among residents, enriching the local arts scene.
Lastly, Rachel O'Connor, a tireless volunteer, dedicates her time to various charitable initiatives. Her commitment to improving the lives of others has been instrumental in creating a strong sense of community within Oak Valley.
Through their unique talents and unwavering dedication, Laura, Kevin, and Rachel have woven themselves into the fabric of Oak Valley, helping to create a vibrant and thriving small town."""

# Prefill for Gemini's response
PREFILL = "<individuals>"

# Print Gemini's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN:")
print(PROMPT)
print("\nASSISTANT TURN:")
print(PREFILL)
print("\n------------------------------------- Gemini's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN:
Silvermist Hollow, a charming village, was home to an extraordinary group of individuals.
Among them was Dr. Liam Patel, a neurosurgeon who revolutionized surgical techniques at the regional medical center.
Olivia Chen was an innovative architect who transformed the village's landscape with her sustainable and breathtaking designs.
The local theater was graced by the enchanting symphonies of Ethan Kovacs, a professionally-trained musician and composer.
Isabella Torres, a self-taught chef with a passion for locally sourced ingredients, created a culinary sensation with her farm-to-table restaurant, which became a must-visit destination for food lovers.
These remarkable individuals, each with their distinct talents, contributed to the vibrant tapestry of life in Silvermist Hollow.
<individuals>
1. Dr. Liam Patel [NEUROSURGEON]
2. Olivia Chen [ARCHITECT]
3. Ethan Kovacs [MISICIAN AND C

---

## Alıştırmalar
- [Alıştırma 7.1 - Örneklerle Email Formatlama](#exercise-71---email-formatting-via-examples)

### Alıştırma 7.1 - Örneklerle Email Formatlama
Alıştırma 6.2'yi tekrar yapacağız, ancak bu sefer `PROMPT`u düzenleyerek Gemini'ın doğru cevabı çıktı olarak almasını sağlamak için "az sayıda" e-posta örneği + uygun sınıflandırma (ve biçimlendirme) kullanacağız. Gemini'ın çıktısındaki *son* harfin kategorinin harfi olmasını istiyoruz.

Her e-posta için hangi harf kategorisinin doğru olduğunu unutursanız, `EMAILS` listesindeki her e-postanın yanındaki yorumlara bakın.

Bunların e-postalar için kategoriler olduğunu unutmayın:										
- (A) Satış öncesi soru
- (B) Kırık veya kusurlu ürün
- (C) Faturalama sorusu
- (D) Diğer (lütfen açıklayınız)								

In [ ]:
# Prompt template with a placeholder for the variable content
PROMPT = """Lütfen bu e-postayı yeşil ya da mavi olarak sınıflandırın: {email}"""

# Prefill for Gemini's response
PREFILL = ""

# Variable content stored as a list
EMAILS = [
    "Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.", # (B) Broken or defective item
    "Can I use my Mixmaster 4000 to mix paint, or is it only meant for mixing food?", # (A) Pre-sale question OR (D) Other (please explain)
    "I HAVE BEEN WAITING 4 MONTHS FOR MY MONTHLY CHARGES TO END AFTER CANCELLING!!  WTF IS GOING ON???", # (C) Billing question
    "How did I get here I am not good with computer.  Halp." # (D) Other (please explain)
]

# Correct categorizations stored as a list of lists to accommodate the possibility of multiple correct categorizations per email
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# Iterate through list of emails
for i,email in enumerate(EMAILS):
    
    # Substitute the email text into the email placeholder variable
    formatted_prompt = PROMPT.format(email=email)
   
    # Get Gemini's response
    response = get_completion(formatted_prompt, prefill=PREFILL)

    # Grade Gemini's response
    grade = any([bool(re.search(ans, response[-1])) for ans in ANSWERS[i]])
    
    # Print Gemini's response
    print("--------------------------- Full prompt with variable substutions ---------------------------")
    print("USER TURN")
    print(formatted_prompt)
    print("\nASSISTANT TURN")
    print(PREFILL)
    print("\n------------------------------------- Gemini's response -------------------------------------")
    print(response)
    print("\n------------------------------------------ GRADING ------------------------------------------")
    print("This exercise has been correctly solved:", grade, "\n\n\n\n\n\n")

❓ If you want a hint, run the cell below!

In [ ]:
from hints import exercise_7_1_hint; print(exercise_7_1_hint)

Hala takıldınız mı? Örnek bir çözüm için aşağıdaki hücreyi çalıştırın.

In [ ]:
from hints import exercise_7_1_solution; print(exercise_7_1_solution)

### Tebrikler!

Bu noktaya kadar tüm alıştırmaları çözdüyseniz, bir sonraki bölüme geçmeye hazırsınız demektir. İyi çalışmalar!

---

## Oyun Alanı

Burası, bu derste gösterilen ipucu örneklerini özgürce deneyebileceğiniz ve Gemini'ın yanıtlarını nasıl etkileyebileceğini görmek için ipuçlarını değiştirebileceğiniz bir alandır.

In [ ]:
# Prompt
PROMPT = "Will Santa bring me presents on Christmas?"

# Print Gemini's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = """Please complete the conversation by writing the next line, speaking as "A".
Q: Is the tooth fairy real?
A: Of course, sweetie. Wrap up your tooth and put it under your pillow tonight. There might be something waiting for you in the morning.
Q: Will Santa bring me presents on Christmas?"""

# Print Gemini's response
print(get_completion(PROMPT))

In [ ]:
# Prompt template with a placeholder for the variable content
PROMPT = """Silvermist Hollow, a charming village, was home to an extraordinary group of individuals.
Among them was Dr. Liam Patel, a neurosurgeon who revolutionized surgical techniques at the regional medical center.
Olivia Chen was an innovative architect who transformed the village's landscape with her sustainable and breathtaking designs.
The local theater was graced by the enchanting symphonies of Ethan Kovacs, a professionally-trained musician and composer.
Isabella Torres, a self-taught chef with a passion for locally sourced ingredients, created a culinary sensation with her farm-to-table restaurant, which became a must-visit destination for food lovers.
These remarkable individuals, each with their distinct talents, contributed to the vibrant tapestry of life in Silvermist Hollow.
<individuals>
1. Dr. Liam Patel [NEUROSURGEON]
2. Olivia Chen [ARCHITECT]
3. Ethan Kovacs [MISICIAN AND COMPOSER]
4. Isabella Torres [CHEF]
</individuals>

At the heart of the town, Chef Oliver Hamilton has transformed the culinary scene with his farm-to-table restaurant, Green Plate. Oliver's dedication to sourcing local, organic ingredients has earned the establishment rave reviews from food critics and locals alike.
Just down the street, you'll find the Riverside Grove Library, where head librarian Elizabeth Chen has worked diligently to create a welcoming and inclusive space for all. Her efforts to expand the library's offerings and establish reading programs for children have had a significant impact on the town's literacy rates.
As you stroll through the charming town square, you'll be captivated by the beautiful murals adorning the walls. These masterpieces are the work of renowned artist, Isabella Torres, whose talent for capturing the essence of Riverside Grove has brought the town to life.
Riverside Grove's athletic achievements are also worth noting, thanks to former Olympic swimmer-turned-coach, Marcus Jenkins. Marcus has used his experience and passion to train the town's youth, leading the Riverside Grove Swim Team to several regional championships.
<individuals>
1. Oliver Hamilton [CHEF]
2. Elizabeth Chen [LIBRARIAN]
3. Isabella Torres [ARTIST]
4. Marcus Jenkins [COACH]
</individuals>

Oak Valley, a charming small town, is home to a remarkable trio of individuals whose skills and dedication have left a lasting impact on the community.
At the town's bustling farmer's market, you'll find Laura Simmons, a passionate organic farmer known for her delicious and sustainably grown produce. Her dedication to promoting healthy eating has inspired the town to embrace a more eco-conscious lifestyle.
In Oak Valley's community center, Kevin Alvarez, a skilled dance instructor, has brought the joy of movement to people of all ages. His inclusive dance classes have fostered a sense of unity and self-expression among residents, enriching the local arts scene.
Lastly, Rachel O'Connor, a tireless volunteer, dedicates her time to various charitable initiatives. Her commitment to improving the lives of others has been instrumental in creating a strong sense of community within Oak Valley.
Through their unique talents and unwavering dedication, Laura, Kevin, and Rachel have woven themselves into the fabric of Oak Valley, helping to create a vibrant and thriving small town."""

# Prefill for Gemini's response
PREFILL = "<individuals>"

# Print Gemini's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN:")
print(PROMPT)
print("\nASSISTANT TURN:")
print(PREFILL)
print("\n------------------------------------- Gemini's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))